In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataframe = pd.read_csv("/content/drive/My Drive/ML Project Taxi Fair/train.csv")

In [ ]:
dataframe.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [ ]:
dataframe.drop(columns=['tripid'], inplace=True)

In [ ]:
def transform(x):
  if x=='correct':
    return 1
  else:
    return 0

dataframe['label'] = dataframe['label'].apply(lambda x : transform(x))

In [ ]:
dataframe.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1
1,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1
2,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1
3,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1
4,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,1


In [ ]:
dataframe.dropna(inplace=True)

In [ ]:
dataframe.drop(columns=['pickup_time','drop_time'], inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

X = dataframe.drop(columns=['label']).values
Y = dataframe['label'].values

scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

In [ ]:
!pip3 install catboost

     |████████████████████████████████| 64.7MB 61kB/s 


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.30, random_state=1)
model = CatBoostClassifier(iterations=1000, devices='0:1')
model.fit(X_train, Y_train, verbose=False)

In [ ]:
predicted = model.predict(X_validation)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(f1_score(predicted, Y_validation))

0.9708963911525029


In [ ]:
test_df = pd.read_csv("/content/drive/My Drive/ML Project Taxi Fair/test.csv")

In [ ]:
ids = test_df['tripid']

X_ = test_df.drop(columns=['pickup_time','drop_time']).values

scaler = StandardScaler()
scaler.fit(X_)

X_ = scaler.transform(X_)
Y_ = model.predict(X_)

In [ ]:
data = {"tripid" : ids, "prediction" : list(Y_)}

sample = pd.DataFrame(data)

In [ ]:
sample.to_csv("/content/drive/My Drive/ML Project Taxi Fair/cat_boost.csv",index=False)